L'objectif ici est d'utiliser les données relatives aux incidents survenus sur le réseau ferroviaire parisien, qui influence probablement le réseau routier. Nous utilisons les données issus de : https://ratpstatus.fr

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../dataset/historique_incidents.csv', sep=";")

In [2]:
df['date de début de l\'incident'] = pd.to_datetime(df['date de début de l\'incident'])
df['date de fin de l\'incident'] = pd.to_datetime(df['date de fin de l\'incident'])

df['statut evenement'].value_counts()

statut evenement
pb    44803
tx    18320
bq    10818
ok       90
Name: count, dtype: int64

In [ ]:
# On veut attribuer un score d'incidents à chaque heure au réseau, on pondère donc par gravité de l'incident et selon l'heure de la journée.
# par exemple des incidents le matin vont avoir un impact sur la circulation alors que la nuit certainement beaucoup moins. 

pond_gravite = {
    "ok": 1,
    "tx": 1,
    "bq": 2,
    "pb": 4
}
df['poids_gravite'] = df['statut evenement'].map(pond_gravite)

def pond_horaire(dt):
    h = dt.hour
    if 6 <= h < 9:
        return 2.0    
    elif 9 <= h < 16:
        return 1.0   
    elif 16 <= h < 20:
        return 2.0    
    elif 20 <= h < 23:
        return 1.2    
    else:
        return 0.5    

rows = []
for _, r in df.iterrows():
    start = r['date de début de l\'incident'].floor('h')
    end = r['date de fin de l\'incident'].ceil('h')
    heures = pd.date_range(start, end, freq='h')
    for h in heures:
        h_fin = h + pd.Timedelta(hours=1)
        overlap = (min(r['date de fin de l\'incident'], h_fin) - max(r['date de début de l\'incident'], h)).total_seconds() / 3600
        if overlap > 0:
            poids_horaire = pond_horaire(h)
            score = r['poids_gravite'] * poids_horaire * overlap
            rows.append({'heure': h, 'score_partiel': score})

df_horaire = pd.DataFrame(rows)

score_horaire = (
    df_horaire
    .groupby('heure', as_index=False)['score_partiel']
    .sum()
    .rename(columns={'score_partiel': 'score_total'})
    .sort_values('heure')
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=score_horaire['heure'],
    y=score_horaire['score_total'],
    mode='lines',
    name='Réseau ferré'
))
fig.show()